In [48]:
import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
from collections import deque

In [49]:
dataset = o3d.data.SampleFountainRGBDImages()

rgbd_images = []
for i in range(len(dataset.depth_paths)):
    depth = o3d.io.read_image(dataset.depth_paths[i])
    color = o3d.io.read_image(dataset.color_paths[i])
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                       color, depth, convert_rgb_to_intensity=False)
    rgbd_images.append(rgbd_image)

camera_trajectory = o3d.io.read_pinhole_camera_trajectory(
                          dataset.keyframe_poses_log_path)
# mesh = o3d.io.read_triangle_mesh(dataset.reconstruction_path)

In [52]:
volume = o3d.pipelines.integration.ScalableTSDFVolume(
    voxel_length=0.01,
    sdf_trunc=0.05,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
)

In [53]:
mesh = volume.extract_triangle_mesh()


In [54]:
def tmp(i):
    w = camera_trajectory.parameters[i].intrinsic.width
    h = camera_trajectory.parameters[i].intrinsic.height
    fx_d = camera_trajectory.parameters[i].intrinsic.intrinsic_matrix[0,0]
    fy_d = camera_trajectory.parameters[i].intrinsic.intrinsic_matrix[1,1]
    cx_d = camera_trajectory.parameters[i].intrinsic.intrinsic_matrix[0,2]
    cy_d = camera_trajectory.parameters[i].intrinsic.intrinsic_matrix[1,2]
    pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(
        w, h, fx_d,fy_d, cx_d, cy_d
    )
    return pinhole_camera_intrinsic

In [55]:

%matplotlib tk
volume.reset()
ts = []

p = [[0,1,0,0],
     [0,0,1,0],
     [0,0,0,1],
     [1,1,1,1]]

# fig = plt.figure(figsize=(10,10))
# ax =  fig.add_subplot(projection='3d')
# ax.set_xlabel('x') # x軸ラベル
# ax.set_ylabel('y') # y軸ラベル
# ax.set_zlabel('z') # z軸ラベル
# ax.set_box_aspect([1, 1, 1])  
for i in range(len(rgbd_images)):
    volume.integrate(
        rgbd_images[i],
        tmp(i),
        camera_trajectory.parameters[i].extrinsic
    )
    # t = camera_trajectory.parameters[i].extrinsic[0:3,3]
    # r = np.copy(camera_trajectory.parameters[i].extrinsic[0:3,0:3])
    # rot = Rotation.from_matrix(r)
    # print(i)
    # print(f"Rot(euler): {rot.as_euler('xyz', degrees=True)}")
    # print(f"t: {t}")
    # ts.append(t)
    res =  np.dot(np.copy(camera_trajectory.parameters[i].extrinsic), p)

    # if i >= 1:
    #     ax.quiver(last_t[0,0],last_t[1,0],last_t[2,0],
    #           res[0,0], res[1,0], res[2,0],
    #           color = "black", length = 0.1,
    #           arrow_length_ratio = 0.2) 
    # ax.quiver(0,0,0,
    #           res[0,0], res[1,0], res[2,0],
    #           color = "black")
    # ax.quiver(res[0,0], res[1,0], res[2,0],
    #           res[0,1], res[1,1], res[2,1],
    #           color = "red", length = 0.1,
    #           arrow_length_ratio = 0.2)
    
    # ax.quiver(res[0,0], res[1,0], res[2,0],
    #           res[0,2], res[1,2], res[2,2],
    #           color = "green", length = 0.1,
    #           arrow_length_ratio = 0.2)
    
    # ax.quiver(res[0,0], res[1,0], res[2,0],
    #           res[0,3], res[1,3], res[2,3],
    #           color = "blue", length = 0.1,
    #           arrow_length_ratio = 0.2)
    last_t = np.copy(res)

ts = np.array(ts)
plt.show()

In [56]:
mesh = volume.extract_triangle_mesh()
# o3d.visualization.draw_geometries([mesh])
# o3d.io.write_triangle_mesh("t.ply", mesh)
o3d.io.write_triangle_mesh("tsdf.ply", mesh)

True

In [57]:
type(camera_trajectory)

open3d.cuda.pybind.camera.PinholeCameraTrajectory

In [58]:
rgbd_images_new  = rgbd_images * 100

In [59]:
camera_trajectory_new = o3d.cuda.pybind.camera.PinholeCameraTrajectory()
camera_trajectory_new.parameters = camera_trajectory.parameters * 100

In [60]:
#https://www.open3d.org/docs/0.9.0/tutorial/Advanced/color_map_optimization.html

# option = o3d.color_map.ColorMapOptimizationOption()
o3d.pipelines.color_map.run_rigid_optimizer(
    mesh, rgbd_images, camera_trajectory,
    o3d.pipelines.color_map.RigidOptimizerOption(maximum_iteration=0))
o3d.io.write_triangle_mesh("rig.ply", mesh)


True

In [61]:
maximum_iteration = 300
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, camera_trajectory = o3d.pipelines.color_map.run_rigid_optimizer(
        mesh, rgbd_images, camera_trajectory,
        o3d.pipelines.color_map.RigidOptimizerOption(maximum_iteration=maximum_iteration))
o3d.io.write_triangle_mesh("rig_300.ply", mesh)

[Open3D DEBUG] [ColorMapOptimization] CreateUtilImagesFromRGBD
[Open3D DEBUG] [ColorMapOptimization] CreateDepthBoundaryMasks
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 0/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 1/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 2/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 3/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 4/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 5/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 6/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 7/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 8/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 9/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 10/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 11/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 12/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 13/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 14/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 15/33
[Open3D DEBUG] [Make

True

In [9]:
maximum_iteration = 300
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, camera_trajectory = o3d.pipelines.color_map.run_non_rigid_optimizer(
        mesh, rgbd_images, camera_trajectory,
        o3d.pipelines.color_map.NonRigidOptimizerOption(
            maximum_iteration=maximum_iteration))

[Open3D DEBUG] [ColorMapOptimization] CreateUtilImagesFromRGBD
[Open3D DEBUG] [ColorMapOptimization] CreateDepthBoundaryMasks
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 0/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 1/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 2/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 3/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 4/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 5/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 6/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 7/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 8/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 9/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 10/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 11/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 12/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 13/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 14/33
[Open3D DEBUG] [MakeDepthMasks] geometry::Image 15/33
[Open3D DEBUG] [Make

In [8]:

# o3d.visualization.draw_geometries([mesh])
o3d.io.write_triangle_mesh("t.ply", mesh)


True